In [ ]:
import numpy as nump 
import math 
import pandas as pan #reading lines from csv file intp an array later 
import os 
import glob # getting the csv files which are stored in the same folder
import tkinter as tkin
from tkinter import filedialog
import matplotlib.pyplot as plt 

import ruptures as rpt
from scipy.signal import savgol_filter

In [ ]:
root=tkin.Tk()
root.withdraw() 
my_working_path= filedialog.askdirectory(title="Choose folder in which other folders with the csv files from the linescan are found:")# opens a window on your computer to select the folder
os.chdir(my_working_path) 
work=os.getcwd()
my_csv_files = glob.glob(os.path.join(work, "*.csv")) # identifies the csv files in this directory
print(my_csv_files)

In [ ]:
for l in range(len(my_csv_files)):
    if "Kopie" not in my_csv_files[l]:
        #later insert code for title search 
        if "Values_Plot" and "alphatub" in my_csv_files[l]:
            print(my_csv_files[l])
            Data_568=pan.read_csv(my_csv_files[l],sep=',',encoding = 'unicode_escape')# using panda for reading the csv file in Dataframe
            plt.plot(Data_568.iloc[:,0],Data_568.iloc[:,1], c="b")# Data_568.iloc[:,0] is x-Data and Data_568.iloc[:,1] is y-data from panda frame 
            array_568=Data_568.to_numpy() # array_568 contains the alphatubulin Data from the Lineprofile over a single MT 
            #print(Data_568)
            #print(array_568)
            #print(Data_568.iloc[:,1])
        elif "Values_Plot" and "acetyl" in my_csv_files[l]:
            print(my_csv_files[l])
            Data_488=pan.read_csv(my_csv_files[l],sep=',', encoding = 'unicode_escape')
            plt.plot(Data_488.iloc[:,0],Data_488.iloc[:,1], c="m")  
            plt.xlabel("Distance [px]")
            plt.ylabel("Intensity [Au]")
            #plt.title("Intensity of Lineprofile of {} stabilized  Microtubule after {} min alphaTAT1 incubation".format(tax, time))
            array_488=Data_488.to_numpy()# array_488 contains the acetylated tubulin Data from the Lineprofile over a single MT 
    #print(my_csv_files[l],reading) #all the csv files in the directory with the same indetifier
print(Data_488)
print(Data_568)
#print(Data_568.columns)



In [ ]:
#converting pixel in µm 0.045400065, 22.0264 pixel/µm
Data_568["Distance_µm"] = Data_568.iloc[:,0].multiply(0.0454)
Data_488["Distance_µm"] = Data_488.iloc[:,0].multiply(0.0454)
position= Data_568.columns.get_loc("Distance_µm")
position_488= Data_488.columns.get_loc("Distance_µm")#
array_488_um=Data_488.to_numpy()
array_568_um=Data_568.to_numpy()
#print(position)
print(position_488)
#Data_568.iloc[:,3] = Data_568.iloc[:,0].multiply(0.0454)
print(Data_568.iloc[:,position])
print(array_488_um)
print(Data_568)


In [ ]:
#detection of changepoint 
#alphatubulin Start and End Microtubule 
algo_new=rpt.Dynp(model="rbf").fit_predict(array_568,n_bkps=4) #works definetly better forn O Minutestax
#algo3=rpt.Dynp(model="rbf").fit_predict(array_568,n_bkps=2)

#Trying to detect patches in acetylation Intensity signal 
algo2=rpt.Pelt(model="rbf").fit_predict(array_488,1.1) # Increase penalty from 1.1 to achieve better fit for acetylated Data 
#  fit_predict(self, signal, pen) # Fit to the signal and return the optimal breakpoints. Helper method to call fit and predict once

# display
rpt.display(array_568[:,1],algo_new,algo_new, computed_chg_pts_linewidth=0.5)

rpt.display(array_488[:,1],algo2, algo2,computed_chg_pts_linewidth=0.5)
#just selects the intensity values for the display function of rupture

#costfunction with penalty 800 aber ohne determination of number of changepoints for acetylated signal patches 
#minimum distance between changepoints 
# ein parameter ändern und dann vergleichen 
plt.show()
print(algo_new)
print(array_568[70:71,1])
print(array_488[70:71,1])

In [ ]:
l_border=algo_new[0]
up_border=algo_new[len(algo_new)-2]
leng_pix=up_border-algo_new[0]
print("lower_border_x:", l_border, "upper_border_x:", up_border, "length in pixel:",leng_pix) 

In [ ]:
#Splitting the array based on the defined borders(=Microtubule start and end) 
n_ar_488_um= array_488_um[l_border:up_border+1,]#slicing the array based on the borders 
end= len(n_ar_488_um[:,0])
leng_um=n_ar_488_um[end-1:end,2]-n_ar_488_um[0:1,2]
#print(n_ar_488_um[end-1:end,0])
l_border_um_48=n_ar_488_um[0:1,2]
up_border_um_48=n_ar_488_um[end-1:end,2]
print("lower_border_x:", l_border_um_48, "upper_border_x:", up_border_um_48, "length in µm:",leng_um )

n_ar_568_um= array_568_um[l_border:up_border+1,]#slicing the array based on the borders 
leng_um_568=n_ar_568_um[end-1:end,2]-n_ar_568_um[0:1,2]
l_border_um_568=n_ar_568_um[0:1,2]
up_border_um_568=n_ar_568_um[end-1:end,2]
print("lower_border_x:", l_border_um_568, "upper_border_x:", up_border_um_568, "length in µm:",leng_um_568 )

#print(n_ar_488_um[:,1])

In [ ]:
#integral = nump.trapz(array_488[:,1], array_488[:,0])
#print(integral) 

integral_alpha = nump.trapz(n_ar_568_um[:,1], n_ar_568_um[:,0])
print(integral_alpha) 

integral_borders= nump.trapz(n_ar_488_um[:,1], n_ar_488_um[:,0])
print(integral_borders) 

#Summe_border=nump.sum(n_ar_488_um[:,1])
#print("Summe b:", Summe_border)
#print(n_ar_488_um[:,1])

Summe=nump.sum(array_488[:,1])
print("Summe 488:", Summe)

Summe_alpha=nump.sum(array_568[:,1])
print("Summe 568:", Summe_alpha)

In [ ]:
# creating values for diagramm 
array_time= [0,5,10,20,60]
value_length_48= leng_um
value_time_48=10 #time 
value_int_48=integral_borders

y48_value= integral_borders/leng_um
print(y48_value)
x48_value= 10 #time 
print("y488:",y48_value[0], "x488:", x48_value)
string48_y= str(y48_value[0])
string48_x=str(x48_value)+","
print(string48_y, string48_x)

y568_value= integral_alpha/leng_um_568
x568_value= 10 #time 
print("y568:",y568_value[0], "x568:", x568_value)
string568_y= str(y568_value[0])
string568_x=str(x568_value)+","
print(string568_y, string568_x)

#sum acetyl/sum alpha 

y_sumAC_sumAL= y48_value/y568_value
string_sumdiv_y= str(y_sumAC_sumAL[0])
print(string_sumdiv_y)

In [ ]:
# saving values sum acetyl/sum alpha normalized over length MT  in a file to open later again and do statistical analysis
root=tkin.Tk()
root.withdraw() 
my_working_path= filedialog.askdirectory(title="Choose folder in which other folders with the csv files from the linescan are found:")# opens a window on your computer to select the folder
os.chdir(my_working_path) 
work=os.getcwd()
f = open('SUMAcetyl_DIV_SUMAlpha_for_timepoints_without_taxol', 'a')

#f = open('AUC_lengthMT_for_timepoints', 'w')

f.write(string48_x)
f.write(string_sumdiv_y+"\n")

#good pracitice
f.close()